In [6]:
#|default_export examples

In [1]:
from titrations.basics import *
from titrations.titrations2 import *

# Beta Blockers

## Dosing Ladder

In [3]:
#|export

metoprolol_succinate = Ingredient("metoprolol succinate")
carvedilol = Ingredient("carvedilol")
bisoprolol = Ingredient("bisoprolol")

beta_blocker_ladder = DosingLadder({
    metoprolol_succinate.name: [
        Medication(metoprolol_succinate, "12.5 mg", "PO", "daily"),
        Medication(metoprolol_succinate, "25 mg", "PO", "daily"),
        Medication(metoprolol_succinate, "50 mg", "PO", "daily"),
        Medication(metoprolol_succinate, "100 mg", "PO", "daily"),
    ],
    carvedilol.name: [
        Medication(carvedilol, "3.125 mg", "PO", "BID"),
        Medication(carvedilol, "6.25 mg", "PO", "BID"),
        Medication(carvedilol, "12.5 mg", "PO", "BID"),
        Medication(carvedilol, "25 mg", "PO", "BID"),
    ],
    bisoprolol.name: [
        Medication(bisoprolol, "1.25 mg", "PO", "BID"),
        Medication(bisoprolol, "2.5 mg", "PO", "BID"),
        Medication(bisoprolol, "5 mg", "PO", "BID"),
        Medication(bisoprolol, "10 mg", "PO", "BID"),
    ]
})

## Rules

In [4]:
#|export
no_pacemaker = Rule('has_pacemaker', 'eq', False)

hypotension = TitrationLimitingRule('SBP', 'lt', 90)
bradycardia = ConditionTitrationLimitingRule('HR' , 'lt', 60, condition=no_pacemaker)
decompensation = TitrationLimitingRule('decompensated', 'eq', True)
symptoms = TitrationLimitingRule('symptomatic', 'eq', True)
av_block = TitrationLimitingRule('av_block', 'eq', True)

## Titrator

In [5]:
#|export
class BetaBlockerTitrator(Titrator):
    dosing_ladder = beta_blocker_ladder
    rules = [
        hypotension,
        bradycardia,
        decompensation,
        symptoms,
        av_block,
    ]

# RAASi

## Dosing Ladder

## Rules

## Titrator

# SGLT2i's

## Dosing Ladder

In [3]:
#|export
dapagliflozin = Ingredient("dapagliflozin")
empagliflozin = Ingredient("empagliflozin")

sglt2i_ladder = DosingLadder({
    dapagliflozin.name: [
        Medication(dapagliflozin, "10 mg", "PO", "daily"),
    ],
    empagliflozin.name: [
        Medication(empagliflozin, "12.5 mg", "PO", "daily"),
    ],
})

## Rules

In [4]:
#|export
low_egfr = TitrationLimitingRule('eGFR', 'lt', 30)
severe_gu_infxns = ClassLimitingRule('severe_gu_infxns', 'eq', True)
has_type_1_diabetes = ClassLimitingRule('has_type_1_diabetes', 'eq', True)
has_type_2_diabetes_on_insulin = NonLimitingRule('has_type_2_diabetes_on_insulin', 'eq', True)

## Titrator

In [7]:
#|export
class SGLT2iTitrator(Titrator):
    dosing_ladder = sglt2i_ladder
    rules = [
        low_egfr,
        severe_gu_infxns,
        has_type_1_diabetes,
        has_type_2_diabetes_on_insulin,
    ]

## Tests

In [19]:
p = Patient(eGFR=20, severe_gu_infxns=False,
            has_type_1_diabetes=False, has_type_2_diabetes_on_insulin=False,
            medications=[Medication(empagliflozin, "12.5 mg", "PO", "daily")])
t = SGLT2iTitrator(p)
t.evaluate()

In [20]:
t.can_advance

False

In [22]:
t.satisfied_rules

[eGFR lt 30]

In [21]:
t.recommended_actions

# MRA's

## Dosing Ladder

## Rules

## Titrator